In [73]:
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LinearR
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold,cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error as mse 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from time import time
import datetime

In [74]:
data = load_boston()
data

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [75]:
X = data.data
y = data.target
Xtrain,Xtest,ytrain,ytest  = train_test_split(X,y,test_size=0.3,random_state=420)


In [76]:
# 训练 使用的是sklearn的api
reg = XGBR(n_estimators=100).fit(Xtrain,ytrain)

In [77]:
reg.predict(Xtest)#传统接口predict

array([ 6.6689262, 22.34918  , 31.052807 , 13.911593 ,  9.467967 ,
       22.658588 , 14.514282 , 15.092699 , 15.293644 , 12.680115 ,
       24.140797 , 35.890083 , 21.573483 , 27.07066  , 19.052658 ,
        9.89033  , 23.386076 , 23.588493 , 23.311466 , 22.401644 ,
       18.98444  , 15.766946 , 25.8352   , 20.193802 , 19.982517 ,
       15.611423 , 22.883228 , 29.838228 , 22.815304 , 16.779034 ,
       37.13194  , 20.133307 , 19.67352  , 23.525528 , 22.845137 ,
       23.87397  , 15.17887  , 23.45934  , 16.685331 , 31.761686 ,
       18.525843 , 22.441063 , 38.48728  , 17.93719  , 15.10122  ,
       28.980541 , 46.363487 , 12.842797 ,  9.618281 , 35.40579  ,
       25.657566 , 20.605602 , 20.800055 , 49.228447 , 31.355848 ,
       29.382515 , 18.911947 , 21.049877 , 16.165169 , 18.098577 ,
       14.659002 , 21.720213 , 19.413454 , 28.932102 , 30.573524 ,
       19.228426 , 20.531511 , 15.666288 , 23.52929  , 19.30554  ,
       28.384985 , 42.83562  , 29.429724 , 23.306015 , 19.7412

In [78]:
reg.score(Xtest,ytest)  # r^2

0.9050988954757183

In [79]:
ypred = reg.predict(Xtest)


In [80]:
y.mean()


22.532806324110677

In [81]:
mse(ytest,ypred)


8.830916470718748

In [82]:
reg.feature_importances_ #树模型的优势之一,能够看模型的重要性分数,可以使用嵌入法(SelectFromModel)进行特征选择 --每个特征对树模型的贡献

array([0.01902167, 0.0042109 , 0.01478317, 0.00553536, 0.02222195,
       0.37914094, 0.01679687, 0.04698721, 0.04073574, 0.05491758,
       0.0668422 , 0.00869463, 0.32011184], dtype=float32)

## 使用五折交叉验证对比RF和线性回归

In [83]:
# xgboost
reg = XGBR(n_estimators=100)
# 交叉验证 并对五折交叉验证的结果(5个值)求均值 ,回归的默认评估指标r^2
cross_val_score(reg,Xtrain,ytrain,cv=5).mean()

0.799506280269948

In [84]:
cross_val_score(reg,Xtrain,ytrain,scoring='neg_mean_squared_error',cv=5).mean()

-16.215644658473447

In [85]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [86]:
# 随机森林
rfr = RFR(n_estimators=100)
cross_val_score(rfr,Xtrain,ytrain,cv=5).mean()


0.7987023468049335

In [87]:
cross_val_score(rfr,Xtrain,ytrain,scoring='neg_mean_squared_error',cv=5).mean()

-17.638664664788728

In [88]:
# 线性回归
lr = LinearR()
cross_val_score(lr,Xtrain,ytrain,cv=5).mean()


0.6835070597278077

In [89]:
cross_val_score(lr,Xtrain,ytrain,scoring='neg_mean_squared_error',cv=5).mean()

-25.349507493648495